# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [1]:
import numpy as np
import regex as re
import pandas as pd
url_1 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv'
url_2 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv'
url_3 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv'

df_1 = pd.read_csv(url_1)
df_2 = pd.read_csv(url_2)
df_3 = pd.read_csv(url_3)

In [2]:
print(df_1.shape, df_2.shape, df_3.shape)
display(df_1.head(1), df_2.head(1), df_3.head(1)) # Getting a glimpse of the three datasets to decide on a strategy to combine them

(4008, 11) (996, 11) (7070, 11)


,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934


,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Total Claim Amount,Policy Type,Vehicle Class
0,GS98873,Arizona,F,Bachelor,323912.47%,16061,88,1/0/00,633.6,Personal Auto,Four-Door Car


,Customer,State,Customer Lifetime Value,Education,Gender,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Total Claim Amount,Vehicle Class
0,SA25987,Washington,3479.137523,High School or Below,M,0,104,0,Personal Auto,499.2,Two-Door Car


In [3]:
def clean_titles(data_frame):
    columns = list(data_frame.columns)
    data_frame.columns = [i.replace('ST', 'state').lower().replace(' ','_').strip() for i in columns]
    return data_frame

In [4]:
# Cleaning column titles to standardise and avoid duplicated columns
clean_titles(df_1)
clean_titles(df_2)
clean_titles(df_3)

,customer,state,customer_lifetime_value,education,gender,income,monthly_premium_auto,number_of_open_complaints,policy_type,total_claim_amount,vehicle_class
0,SA25987,Washington,3479.137523,High School or Below,M,0,104,0,Personal Auto,499.200000,Two-Door Car
1,TB86706,Arizona,2502.637401,Master,M,0,66,0,Personal Auto,3.468912,Two-Door Car
2,ZL73902,Nevada,3265.156348,Bachelor,F,25820,82,0,Personal Auto,393.600000,Four-Door Car
3,KX23516,California,4455.843406,High School or Below,F,0,121,0,Personal Auto,699.615192,SUV
4,FN77294,California,7704.958480,High School or Below,M,30366,101,2,Personal Auto,484.800000,SUV
...,...,...,...,...,...,...,...,...,...,...,...
7065,LA72316,California,23405.987980,Bachelor,M,71941,73,0,Personal Auto,198.234764,Four-Door Car
7066,PK87824,California,3096.511217,College,F,21604,79,0,Corporate Auto,379.200000,Four-Door Car
7067,TD14365,California,8163.890428,Bachelor,M,0,85,3,Corporate Auto,790.784983,Four-Door Car
7068,UP19263,California,7524.442436,College,M,21941,96,0,Personal Auto,691.200000,Four-Door Car


In [5]:
# Concatenating the DataFrames and resettign the index
df = pd.concat([df_1, df_2, df_3], ignore_index=True)

In [6]:
df['gender'] = df['gender'].str.lower().str.strip()
# Define a function to normalise different gender inputs 
def normalise_gender(value):
    if re.search(r'\b(m|male)\b', value):   # searches values for males
        return 'M'
    elif re.search(r'\b(f|female|femal)\b', value):   # searches values for females
        return 'F'
    
# Apply the function normalise_gender to the gender series ignoring NaN values and overwrite the series with the new values
df['gender'] = df['gender'].map(normalise_gender, na_action='ignore')

# Lowercasing and removing whitespaces from the state series
df['state'] = df['state'].str.lower().str.strip()

# Define a function to normalise different state inputs 
def normalise_state(value):
    if re.search(r'\b(oregon)\b', value):   # searches values for oregon
        return 'Oregon'
    elif re.search(r'\b(california|cali)\b', value):   # searches values for california
        return 'California'
    elif re.search(r'\b(arizona|az)\b', value):   # searches values for arizona
        return 'Arizona'
    elif re.search(r'\b(nevada)\b', value):   # searches values for nevada
        return 'Nevada'
    elif re.search(r'\b(washington|wa)\b', value):   # searches values for washington
        return 'Washington'
# Apply the function normalise_state to the state series ignoring NaN values and overwrite the series with the new values
df['state'] = df['state'].map(normalise_state, na_action='ignore')

# Removing whitespaces from the education series
df['education'] = df['education'].str.strip()

# Define a function to normalise different education inputs 
def normalise_education(value):
    if re.search(r'\b(Bachelors|Bachelor)\b', value):  
        return 'Bachelor'
    elif re.search(r'\b(College)\b', value):  
        return 'College'
    elif re.search(r'\b(High School or Below)\b', value):  
        return 'High School or Below'
    elif re.search(r'\b(Master)\b', value):  
        return 'Master'
    elif re.search(r'\b(Doctor)\b', value):  
        return 'PhD'
# Apply the function normalise_education to the education series ignoring NaN values and overwrite the series with the new values
df['education'] = df['education'].map(normalise_education, na_action='ignore')

# Lowercasing and removing whitespaces from the vehicle_class series
df['vehicle_class'] = df['vehicle_class'].str.lower().str.strip()

# Define a function to normalise different vehicle inputs 
def normalise_vehicle(value):
    if re.search(r'\b(four-door car)\b', value):  
        return 'Four-Door Car'
    elif re.search(r'\b(two-door car)\b', value):  
        return 'Two-Door Car'
    elif re.search(r'\b(sports car|luxury suv|luxury car)\b', value):  
        return 'Luxury'
    elif re.search(r'\b(suv)\b', value):  
        return 'SUV'
    
# Apply the function normalise_vehicle to the vehicle_class series ignoring NaN values and overwrite the series with the new values
df['vehicle_class'] = df['vehicle_class'].map(normalise_vehicle, na_action='ignore')

# Removing '%' and casting as a float
df['customer_lifetime_value'] = df['customer_lifetime_value'].str.replace('%','').astype(float)

# Define a function to normalise complaints
def split_complaints(value):
    try:
        if isinstance(value, str):
            value_split = value.split('/')
            return value_split[1]
    except AttributeError:
        return value
    
df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(split_complaints)

In [ ]:
df.drop_duplicates(inplace=True) # remove all duplicates
df.reset_index(inplace=True) # reset the index

In [31]:
df.isnull().sum()

level_0                         0
index                           0
customer                        1
state                           1
gender                        123
education                       1
customer_lifetime_value      7078
income                          1
monthly_premium_auto            1
number_of_open_complaints    7071
policy_type                     1
vehicle_class                   1
total_claim_amount              1
dtype: int64

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [83]:
url_4 = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv'

df_4 = pd.read_csv(url_4)
display(df_4.head(1))
print(df_4.columns)

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.21696,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.8,Four-Door Car,Medsize,A,2


Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')


1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [82]:
df_sales = (df_4.pivot_table(index='sales_channel', values='customer_lifetime_value', aggfunc='sum')).round(2)
print('Total revenue of each department:\n-----------------------------------')
display(df_sales)
print('Percentage-wise:\n-----------------------------------')
df_sales_pct = (df_sales/df_sales.sum()*100).round()
display(df_sales_pct)

Total revenue of each department:
-----------------------------------


,customer_lifetime_value
sales_channel,
Agent,33057887.85
Branch,24359201.21
Call Center,17364288.37
Web,12697632.90


Percentage-wise:
-----------------------------------


,customer_lifetime_value
sales_channel,
Agent,38.0
Branch,28.0
Call Center,20.0
Web,15.0


In [ ]:
## The highest revenue came from agents (~38% of sales). The lowest revenue came from online clients (15%).

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [101]:
df_value_gender = df_4.pivot_table(index='education', columns='gender', values='customer_lifetime_value', aggfunc='mean').round(2)
df_value_gender

gender,F,M
education,,
Bachelor,7874.27,7703.60
College,7748.82,8052.46
Doctor,7328.51,7415.33
High School or Below,8675.22,8149.69
Master,8157.05,8168.83


In [ ]:
# There seems to be a trend for level of education versus revenue obtained from customers. Customers with higher level of education
# appear to have a lower customer lifetime value. There are also differences between male and female customers: female customers 
# appear to have a higher customer lifetime value, but only at lower levels of education (i.e. high school or below), while this gap
# disappears at higher education levels.

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [173]:
df_pivot = (df_4.pivot_table(index=['month', 'policy_type'], values='number_of_open_complaints', aggfunc='sum')).round()
df_pivot

number_of_open_complaints
month policy_type                              
1     Corporate Auto                      443.0
      Personal Auto                      1728.0
      Special Auto                         87.0
2     Corporate Auto                      385.0
      Personal Auto                      1454.0
      Special Auto                         95.0